Checks whether the target database `AppliedDatabaseTechnologyFinalProject` exists and creates it if not.

In [27]:
import mysql.connector
import warnings
warnings.filterwarnings("ignore", category=UserWarning)

# Connect to MySQL server without specifying a database
config = {
    'user': 'root',
    'password': 'root',
    'host': 'localhost',
    'unix_socket': '/Applications/MAMP/tmp/mysql/mysql.sock',
    'raise_on_warnings': False
}

try:
    conn = mysql.connector.connect(**config)
    cursor = conn.cursor()

    # Create the database if it does not already exist
    cursor.execute("CREATE DATABASE IF NOT EXISTS AppliedDatabaseTechnologyFinalProject")
    print("Database check complete.")

except mysql.connector.Error as err:
    print("Database connection or creation error:", err)

# Reconnect to use the specific project database
config['database'] = 'AppliedDatabaseTechnologyFinalProject'
conn = mysql.connector.connect(**config)
cursor = conn.cursor()


Database check complete.


Cell 3: Create Database Tables

This cell creates the four main tables used in the project:

- Properties: Stores information about each property including location, size, and estimated value.
- Sales: Records property sales data and links each sale to a property.
- Neighbors: Represents relationships between nearby properties based on location.
- Regional_Prices: Contains average pricing data by zip code and date.

Each table includes primary keys and foreign keys to maintain data structure and consistency.


In [3]:
# SQL statements to create each table
create_properties = """
CREATE TABLE IF NOT EXISTS Properties (
    property_id BIGINT PRIMARY KEY,
    zip_code VARCHAR(10) NOT NULL,
    lat DOUBLE NOT NULL,
    lon DOUBLE NOT NULL,
    sqft INT,
    bedrooms INT,
    bathrooms FLOAT,
    year_built INT,
    valuation DOUBLE NOT NULL
);
"""

create_sales = """
CREATE TABLE IF NOT EXISTS Sales (
    sale_id INT AUTO_INCREMENT PRIMARY KEY,
    property_id BIGINT NOT NULL,
    sale_price DOUBLE NOT NULL,
    sale_date DATE NOT NULL,
    FOREIGN KEY (property_id) REFERENCES Properties(property_id)
);
"""

create_neighbors = """
CREATE TABLE IF NOT EXISTS Neighbors (
    property_id BIGINT NOT NULL,
    neighbor_id BIGINT NOT NULL,
    distance_miles DOUBLE NOT NULL CHECK (distance_miles >= 0),
    PRIMARY KEY (property_id, neighbor_id),
    FOREIGN KEY (property_id) REFERENCES Properties(property_id),
    FOREIGN KEY (neighbor_id) REFERENCES Properties(property_id)
);
"""

create_regional_prices = """
CREATE TABLE IF NOT EXISTS Regional_Prices (
    zip_code VARCHAR(10) NOT NULL,
    date DATE NOT NULL,
    avg_price DOUBLE NOT NULL CHECK (avg_price >= 0),
    avg_price_per_sqft DOUBLE,
    PRIMARY KEY (zip_code, date)
);
"""

# Execute all table creation statements
for ddl in [create_properties, create_sales, create_neighbors, create_regional_prices]:
    cursor.execute(ddl)

conn.commit()
print("All tables have been created successfully.")

All tables have been created successfully.


Cell 4: Reset Table Data

This cell clears all data from the database tables by truncating them. Foreign key checks are temporarily disabled to prevent dependency errors during truncation. After all tables are cleared, constraints are re-enabled. This ensures a clean state for fresh data inserts.

In [14]:
# Disable foreign key checks to allow truncating all tables
cursor.execute("SET FOREIGN_KEY_CHECKS = 0")

# Truncate all tables (order doesn't matter when FK checks are off)
cursor.execute("TRUNCATE TABLE Sales")
cursor.execute("TRUNCATE TABLE Neighbors")
cursor.execute("TRUNCATE TABLE Regional_Prices")
cursor.execute("TRUNCATE TABLE Properties")

# Re-enable foreign key checks
cursor.execute("SET FOREIGN_KEY_CHECKS = 1")

conn.commit()
print("All tables truncated and constraints re-enabled.")

All tables truncated and constraints re-enabled.


Cell 5: Load and Prepare Data

This cell loads the raw CSV dataset and prepares two pandas DataFrames:

- properties_df: Contains property-level details including location, structure, and valuation.
- sales_df: Contains sale price and date information for each property.

It also converts sale dates to standard date format and previews the top 20 rows from each DataFrame.


In [15]:
import pandas as pd

# Load your dataset
csv_path = '/Users/michaeljohnson/Documents/AppliedDatabaseTechnology/FinalProject/Data/kc_house_data.csv'
df = pd.read_csv(csv_path)

# Create Properties DataFrame
properties_df = df[[
    'id', 'zipcode', 'lat', 'long', 'sqft_living', 'bedrooms', 'bathrooms', 'yr_built', 'price'
]].copy()

properties_df.columns = [
    'property_id', 'zip_code', 'lat', 'lon', 'sqft', 'bedrooms', 'bathrooms', 'year_built', 'valuation'
]

# Preview top 20 rows from Properties DataFrame
print("Top 20 rows from Properties DataFrame:")
display(properties_df.head(20))

# Prepare Sales DataFrame
sales_df = df[['id', 'price', 'date']].copy()
sales_df.columns = ['property_id', 'sale_price', 'sale_date']
sales_df['sale_date'] = pd.to_datetime(sales_df['sale_date']).dt.date

# Preview top 20 rows from Sales DataFrame
print("Top 20 rows from Sales DataFrame:")
display(sales_df.head(20))


Top 20 rows from Properties DataFrame:


,property_id,zip_code,lat,lon,sqft,bedrooms,bathrooms,year_built,valuation
0,7129300520,98178,47.5112,-122.257,1180,3,1.00,1955,221900.0
1,6414100192,98125,47.7210,-122.319,2570,3,2.25,1951,538000.0
2,5631500400,98028,47.7379,-122.233,770,2,1.00,1933,180000.0
3,2487200875,98136,47.5208,-122.393,1960,4,3.00,1965,604000.0
4,1954400510,98074,47.6168,-122.045,1680,3,2.00,1987,510000.0
5,7237550310,98053,47.6561,-122.005,5420,4,4.50,2001,1225000.0
6,1321400060,98003,47.3097,-122.327,1715,3,2.25,1995,257500.0
7,2008000270,98198,47.4095,-122.315,1060,3,1.50,1963,291850.0
8,2414600126,98146,47.5123,-122.337,1780,3,1.00,1960,229500.0
9,3793500160,98038,47.3684,-122.031,1890,3,2.50,2003,323000.0


Top 20 rows from Sales DataFrame:


,property_id,sale_price,sale_date
0,7129300520,221900.0,2014-10-13
1,6414100192,538000.0,2014-12-09
2,5631500400,180000.0,2015-02-25
3,2487200875,604000.0,2014-12-09
4,1954400510,510000.0,2015-02-18
5,7237550310,1225000.0,2014-05-12
6,1321400060,257500.0,2014-06-27
7,2008000270,291850.0,2015-01-15
8,2414600126,229500.0,2015-04-15
9,3793500160,323000.0,2015-03-12


Cell 6: Insert Data into Properties Table

This cell inserts the cleaned property data into the Properties table. It uses a REPLACE INTO statement to avoid primary key conflicts by overwriting any existing records with the same property_id. After the insert, it previews the first 20 rows to confirm the data was loaded correctly.


In [16]:
# Use REPLACE to overwrite existing property_id if needed
insert_property = """
REPLACE INTO Properties
(property_id, zip_code, lat, lon, sqft, bedrooms, bathrooms, year_built, valuation)
VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)
"""

property_data = list(properties_df.itertuples(index=False, name=None))
print("Rows to insert:", len(property_data))

cursor.executemany(insert_property, property_data)
conn.commit()

# Preview
query = "SELECT * FROM Properties LIMIT 20"
properties_preview = pd.read_sql(query, conn)
properties_preview


Rows to insert: 21613


/var/folders/kw/22rb_dln7zv_jyth2ccrcwm40000gn/T/ipykernel_1738/635738413.py:16: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  properties_preview = pd.read_sql(query, conn)


,property_id,zip_code,lat,lon,sqft,bedrooms,bathrooms,year_built,valuation
0,1000102,98002,47.3262,-122.214,2400,6,3.00,1991,300000.0
1,1200019,98166,47.4444,-122.351,2060,4,1.75,1947,647500.0
2,1200021,98166,47.4434,-122.347,1460,3,1.00,1952,400000.0
3,2800031,98168,47.4783,-122.265,1430,3,1.00,1930,235000.0
4,3600057,98144,47.5803,-122.294,1650,4,2.00,1951,402500.0
5,3600072,98144,47.5801,-122.294,2220,4,2.75,1951,680000.0
6,3800008,98178,47.4938,-122.262,1990,5,1.50,1960,178000.0
7,5200087,98108,47.5423,-122.302,2540,4,2.50,2005,487000.0
8,6200017,98032,47.4023,-122.273,1340,3,1.00,1945,281000.0
9,7200080,98055,47.4836,-122.214,1980,4,2.00,1924,239000.0


Cell 7: Insert Data into Sales Table

This cell inserts sale transaction data into the Sales table. Each row links a property to its sale price and date. The insert uses a batch method for efficiency and does not overwrite existing entries. A preview confirms that the data was inserted correctly.


In [17]:
# Insert data into Sales table
insert_sale = """
INSERT INTO Sales
(property_id, sale_price, sale_date)
VALUES (%s, %s, %s)
"""

sale_data = list(sales_df.itertuples(index=False, name=None))
print("Rows to insert:", len(sale_data))

cursor.executemany(insert_sale, sale_data)
conn.commit()

# Preview Sales table
query = "SELECT * FROM Sales LIMIT 20"
sales_preview = pd.read_sql(query, conn)
sales_preview

Rows to insert: 21613


/var/folders/kw/22rb_dln7zv_jyth2ccrcwm40000gn/T/ipykernel_1738/2444586917.py:16: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  sales_preview = pd.read_sql(query, conn)


,sale_id,property_id,sale_price,sale_date
0,1,7129300520,221900.0,2014-10-13
1,2,6414100192,538000.0,2014-12-09
2,3,5631500400,180000.0,2015-02-25
3,4,2487200875,604000.0,2014-12-09
4,5,1954400510,510000.0,2015-02-18
5,6,7237550310,1225000.0,2014-05-12
6,7,1321400060,257500.0,2014-06-27
7,8,2008000270,291850.0,2015-01-15
8,9,2414600126,229500.0,2015-04-15
9,10,3793500160,323000.0,2015-03-12


Cell 8: Generate and Insert Regional Price Data

This cell generates average sale price and price per square foot for each ZIP code and month combination. It merges sales data with property details, groups it by ZIP and month, calculates the metrics, and inserts the result into the Regional_Prices table. This data supports valuation comparisons in later queries.

In [18]:
# Step 1: Merge Sales with Properties to get ZIP, sale_price, sqft
merged_df = sales_df.merge(
    properties_df[['property_id', 'zip_code', 'sqft']],
    on='property_id',
    how='inner'
)

# Step 2: Convert sale_date to first day of the month
merged_df['sale_month'] = pd.to_datetime(merged_df['sale_date']).values.astype('datetime64[M]')
merged_df['sale_month'] = merged_df['sale_month'].dt.strftime('%Y-%m-01')

# Step 3: Compute average price and average price per sqft by ZIP and month
grouped = merged_df.groupby(['zip_code', 'sale_month']).agg(
    avg_price=('sale_price', 'mean'),
    avg_price_per_sqft=('sale_price', lambda x: (x / merged_df.loc[x.index, 'sqft']).mean())
).reset_index()

# Step 4: Prepare data for MySQL insert
regional_price_data = list(grouped.itertuples(index=False, name=None))

# Step 5: Clear existing table data
cursor.execute("DELETE FROM Regional_Prices")

# Step 6: Insert data
insert_regional = """
INSERT INTO Regional_Prices
(zip_code, date, avg_price, avg_price_per_sqft)
VALUES (%s, %s, %s, %s)
"""

cursor.executemany(insert_regional, regional_price_data)
conn.commit()
print("Regional_Prices table populated.")

# Step 7: Preview top 20
query = "SELECT * FROM Regional_Prices LIMIT 20"
regional_preview = pd.read_sql(query, conn)
regional_preview


Regional_Prices table populated.


/var/folders/kw/22rb_dln7zv_jyth2ccrcwm40000gn/T/ipykernel_1738/1753632579.py:37: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  regional_preview = pd.read_sql(query, conn)


,zip_code,date,avg_price,avg_price_per_sqft
0,98001,2014-05-01,242414.696970,145.234465
1,98001,2014-06-01,268583.580645,155.770646
2,98001,2014-07-01,276466.666667,142.209185
3,98001,2014-08-01,277040.384615,149.761057
4,98001,2014-09-01,298152.257143,144.114862
5,98001,2014-10-01,281170.689655,140.238802
6,98001,2014-11-01,264087.826087,164.517594
7,98001,2014-12-01,277294.600000,157.345756
8,98001,2015-01-01,278641.000000,143.155594
9,98001,2015-02-01,292197.368421,165.032428


Cell 9: Generate and Insert Neighbor Relationships

This cell populates the Neighbors table by identifying properties that share the same ZIP code. To avoid excessive data, it limits each property to a maximum of five random neighbors. All distances are assigned a fixed value of 0.5 miles. The results are inserted in batches to handle large volumes efficiently and prevent database packet size errors.

In [19]:
import numpy as np
# Join properties on ZIP and exclude self-joins
same_zip = properties_df[['property_id', 'zip_code']].merge(
    properties_df[['property_id', 'zip_code']],
    on='zip_code'
)

# Filter out self-pairs
same_zip = same_zip[same_zip['property_id_x'] != same_zip['property_id_y']]

# For each property, keep only 5 neighbors max
same_zip['rand'] = np.random.rand(len(same_zip))
same_zip = same_zip.sort_values(['property_id_x', 'rand']).groupby('property_id_x').head(5)

# Assign fixed distance
same_zip['distance_miles'] = 0.5

# Rename and format
same_zip = same_zip[['property_id_x', 'property_id_y', 'distance_miles']]
same_zip.columns = ['property_id', 'neighbor_id', 'distance_miles']

# Drop duplicates
same_zip.drop_duplicates(subset=['property_id', 'neighbor_id'], inplace=True)

# Convert to tuples
neighbor_data = list(same_zip.itertuples(index=False, name=None))

# Clear and insert in batches
cursor.execute("DELETE FROM Neighbors")

def chunk_list(data, size):
    for i in range(0, len(data), size):
        yield data[i:i + size]

insert_neighbors = """
INSERT INTO Neighbors (property_id, neighbor_id, distance_miles)
VALUES (%s, %s, %s)
"""

for chunk in chunk_list(neighbor_data, 1000):
    cursor.executemany(insert_neighbors, chunk)
    conn.commit()

print("Inserted", len(neighbor_data), "neighbors.")

# Preview
query = "SELECT * FROM Neighbors LIMIT 20"
neighbors_preview = pd.read_sql(query, conn)
neighbors_preview


Inserted 107165 neighbors.


/var/folders/kw/22rb_dln7zv_jyth2ccrcwm40000gn/T/ipykernel_1738/1822891.py:48: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  neighbors_preview = pd.read_sql(query, conn)


,property_id,neighbor_id,distance_miles
0,1000102,835000055,0.5
1,1000102,3224800075,0.5
2,1000102,7116000350,0.5
3,1000102,7116500925,0.5
4,1000102,7335400215,0.5
5,1200019,616000140,0.5
6,1200019,1219000473,0.5
7,1200019,2968801085,0.5
8,1200019,4197400043,0.5
9,1200019,5072200040,0.5


Cell 10: Data Validation and Profiling

This cell checks the database content against the original source dataset kc_house_data.csv. It performs the following steps:

- Row Counts: Displays how many rows are in each of the four database tables.
- Deduplication Check: Compares the number of unique property IDs in the source file to the number of records in the Properties table.
- Balancing Check: Verifies whether the number of Sales records matches the total rows in the source file.
- Summary Statistics:
  - For the Properties table: square footage, bedrooms, bathrooms, and valuation
  - For the Sales table: sale price
- Regional Coverage: Shows the number of unique zip codes and months in the Regional_Prices table.
- Neighbors Table: Displays the total number of neighbor relationships created.

The output uses two decimal places for all numeric summaries. A pass or warning message is printed to show whether the database aligns with the source data.


In [28]:
import pandas as pd
pd.set_option('display.float_format', '{:.2f}'.format)  # Format numbers to 2 decimal places

# Reload source file
source_df = pd.read_csv(csv_path)

# MySQL row counts
tables = ['Properties', 'Sales', 'Regional_Prices', 'Neighbors']
row_counts = {}

for table in tables:
    query = f"SELECT COUNT(*) AS count FROM {table}"
    result = pd.read_sql(query, conn)
    row_counts[table] = result.iloc[0, 0]

print("Row Counts in MySQL Tables:")
for table, count in row_counts.items():
    print(f"{table}: {count} rows")

# Count unique property IDs from the source CSV
unique_properties = source_df['id'].nunique()

print("\nSource CSV (kc_house_data.csv):")
print(f"Raw rows in source:            {len(source_df)}")
print(f"Unique property_id values:     {unique_properties}")

print("\nData Balancing Check:")
print(f"Properties table rows:         {row_counts['Properties']}")
print(f"Sales table rows:              {row_counts['Sales']}")

# Evaluate balance
if row_counts['Properties'] == unique_properties and row_counts['Sales'] == len(source_df):
    print("\nPASS: Properties and Sales tables match the source dataset after deduplication.")
else:
    print("\nWARNING: Table counts do not fully align with the source.")
    if row_counts['Properties'] != unique_properties:
        print("- Properties row count does not match number of unique property IDs.")
    if row_counts['Sales'] != len(source_df):
        print("- Sales row count does not match total source rows.")

# Summary statistics for Properties
query = "SELECT sqft, bedrooms, bathrooms, valuation FROM Properties"
prop_stats = pd.read_sql(query, conn)
print("\nProperties Summary Statistics:")
display(prop_stats.describe().round(2))

# Summary statistics for Sales
query = "SELECT sale_price FROM Sales"
sale_stats = pd.read_sql(query, conn)
print("\nSales Summary Statistics:")
display(sale_stats.describe().round(2))

# Coverage in Regional_Prices
query = "SELECT COUNT(DISTINCT zip_code) AS zip_count, COUNT(DISTINCT date) AS month_count FROM Regional_Prices"
coverage = pd.read_sql(query, conn)
print("\nRegional Prices Coverage:")
display(coverage)

# Count total Neighbors
query = "SELECT COUNT(*) AS total_pairs FROM Neighbors"
neighbors_count = pd.read_sql(query, conn)
print("\nTotal Neighbor Relationships:")
display(neighbors_count)


Row Counts in MySQL Tables:
Properties: 21436 rows
Sales: 21613 rows
Regional_Prices: 904 rows
Neighbors: 107165 rows

Source CSV (kc_house_data.csv):
Raw rows in source:            21613
Unique property_id values:     21436

Data Balancing Check:
Properties table rows:         21436
Sales table rows:              21613

PASS: Properties and Sales tables match the source dataset after deduplication.

Properties Summary Statistics:


,sqft,bedrooms,bathrooms,valuation
count,21436.00,21436.00,21436.00,21436.00
mean,2082.70,3.37,2.12,541649.96
std,919.15,0.93,0.77,367314.93
min,290.00,0.00,0.00,75000.00
25%,1430.00,3.00,1.75,324866.00
50%,1920.00,3.00,2.25,450000.00
75%,2550.00,4.00,2.50,645000.00
max,13540.00,33.00,8.00,7700000.00



Sales Summary Statistics:


,sale_price
count,21613.00
mean,540088.14
std,367127.20
min,75000.00
25%,321950.00
50%,450000.00
75%,645000.00
max,7700000.00



Regional Prices Coverage:


,zip_count,month_count
0,70,13



Total Neighbor Relationships:


,total_pairs
0,107165


In [29]:
# Join Properties with latest Regional_Prices for matching ZIP/month
# Use a sample month from the Regional_Prices table
latest_month = pd.read_sql("SELECT MAX(date) as latest_date FROM Regional_Prices", conn).iloc[0]['latest_date']

query = f"""
SELECT p.property_id, p.zip_code, p.valuation, r.avg_price,
       ROUND((r.avg_price - p.valuation) / r.avg_price * 100, 2) AS undervalued_percent
FROM Properties p
JOIN Regional_Prices r
  ON p.zip_code = r.zip_code
WHERE r.date = '{latest_month}' AND p.valuation < r.avg_price
ORDER BY undervalued_percent DESC
LIMIT 20
"""

undervalued_df = pd.read_sql(query, conn)
print(f"Undervalued Properties for {latest_month}:")
display(undervalued_df)


Undervalued Properties for 2015-05-01:


,property_id,zip_code,valuation,avg_price,undervalued_percent
0,5016003230,98112,169317.00,1540000.00,89.01
1,1049010620,98034,90000.00,594218.18,84.85
2,1424059130,98006,247500.00,1587864.00,84.41
3,2202500080,98006,248000.00,1587864.00,84.38
4,8658300340,98014,80000.00,487500.00,83.59
5,6071800480,98006,271950.00,1587864.00,82.87
6,2206500105,98006,290000.00,1587864.00,81.74
7,8965450190,98006,295000.00,1587864.00,81.42
8,3421079032,98022,75000.00,396158.17,81.07
9,1238500451,98033,130000.00,654385.87,80.13
